In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/indian-startup-funding/startup_funding.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.drop(columns=['Remarks'],inplace=True)
# since this column has only 419 non-null values compared to 2625 null hence only about 15% data so we'll drop the column .

In [ ]:
df.set_index('Sr No',inplace=True)

In [ ]:
df.head()

In [ ]:
df.rename(columns={
    'Date dd/mm/yyyy':'date',
    'Startup Name':'startup',
    'Industry Vertical':'vertical',
    'SubVertical':'subvertical',
    'City  Location':'city',
    'Investors Name':'investors',
    'InvestmentnType':'round',
    'Amount in USD':'amount'    
    
},inplace=True)

In [ ]:
df.head()

In [ ]:
df['amount'] = df['amount'].fillna('0')
# since we want amount in rupees so replacing null to 0 .

In [ ]:
df['amount'] = df['amount'].str.replace(',','')
df['amount'] = df['amount'].str.replace('undisclosed','0')
df['amount'] = df['amount'].str.replace('unknown','0')
df['amount'] = df['amount'].str.replace('Undisclosed','0')

In [ ]:
df = df[df['amount'].str.isdigit()]

In [ ]:
df['amount'] = df['amount'].astype('float')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
def to_inr(dollar):
    inr = dollar * 88.5
    return inr/10000000

In [ ]:
df['amount'] = df['amount'].apply(to_inr)
# converting dollar to inr

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'],errors='coerce')
# since many dates have issue in format so skip those rows for now

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.dropna(subset=['date','startup','vertical','city','investors','round','amount'])

In [ ]:
df.info()

- so now only subvertical column has null values

In [ ]:
df.to_csv('startup_cleaned.csv',index=False)

In [ ]:
df.head()

In [ ]:
sorted(set(df['investors'].str.split(',').sum()))
# now we have split the name of investors for each startup and also removed duplicates . 

In [ ]:
df[df['investors'].str.contains(' IDG Ventures')].head()[['date','startup','vertical','city','round','amount']]

In [ ]:
df[df['investors'].str.contains(' IDG Ventures')].groupby('startup')['amount'].sum().sort_values(ascending=False)

In [ ]:
df[df['investors'].str.contains(' IDG Ventures')].groupby('city')['amount'].sum().plot(kind='pie')

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df.head()

In [ ]:
df[df['investors'].str.contains(' IDG Ventures')].groupby('year')['amount'].sum().plot()

In [ ]:
round(df['amount'].max())

In [ ]:
df[df['amount'] == 32175]

In [ ]:
df['startup'].nunique()

In [ ]:
df['month'] = df['date'].dt.month

In [ ]:
df

In [ ]:
df.groupby(['year','month'])['startup'].count().reset_index()

In [ ]:
temp_df = df.groupby(['year','month'])['amount'].sum().reset_index()

In [ ]:
temp_df['x_axis'] = temp_df['month'].astype('str') + '-' + temp_df['year'].astype('str')

In [ ]:
temp_df[['amount','x_axis']]